In [ ]:
# default_exp core

# pandas temporal query language





In [0]:
#export

import re
import numpy as np
import pandas as pd
from functools import singledispatch

# General helper functions

## Info

In [0]:
#export
class Info():
  """
  A class to store information about the data and results from analysis
  """
  def __init__(self):
      self.evaluated = {}

## memory

In [0]:
#export
def memory(info, func, expr):
  """
  checks if the function has been called with the same argument previously and
  if so, returns the same results instead of running the function again

  args:
    -
  """
  rows=None
  if info:
    if func in info.evaluated:
      if expr in info.evaluated[func]:
        rows = info.evaluated[func][expr]
    else:
      info.evaluated[func] = {}
  else:
    info = Info()
    info.evaluated[func] = {}
  return info, rows

## listify

In [0]:
#export
def listify(string_or_list):
    """
    return a list if the input is a string, if not: returns the input as it was

    Args:
        string_or_list (str or any):

    Returns:
        A list if the input is a string, if not: returns the input as it was

    Note:
        - allows user to use a string as an argument instead of single lists
        - cols='icd10' is allowed instead of cols=['icd10']
        - cols='icd10' is transformed to cols=['icd10'] by this function

    """
    if isinstance(string_or_list, str):
        string_or_list = [string_or_list]
    return string_or_list

## unique

In [0]:
#export
# A function to identify all unique values in one or more columns
# with one or multiple codes in each cell

def unique(df, cols=None, sep=None, all_str=True):
  """
  Lists unique values from one or more columns

  sep (str): separator if cells have multiple values
  all_str (bool): converts all values to strings

  unique(df=df, cols='inpatient', sep=',')
  """
  # if no column(s) are specified, find unique values in whole dataframe
  if cols==None:
    cols=list(df.columns)
  cols = listify(cols)

  # multiple values with separator in cells
  if sep:
    all_unique=set()

    for col in cols:
      new_unique = set(df[col].str.cat(sep=',').split(','))
      all_unique.update(new_unique)
  # single valued cells
  else:
    all_unique = pd.unique(df[cols].values.ravel('K'))

  # if need to make sure all elements are strings without surrounding spaces
  if all_str:
    all_unique=[str(value).strip() for value in all_unique]

  return all_unique

In [0]:
#unique(df=df, cols='codes', sep=',')

## del dot and zero

In [0]:
#export
def del_dot(code):
  if isinstance(code, str):
    return code.replace('.','')
  else:
    codes = [c.replace('.','') for c in code]
  return codes

def del_zero(code, left=True, right=False):
  if isinstance(codes, str):
    codes=[code]
  if left:
    codes = [c.lstrip('0') for c in code]
  if right:
    codes = [c.rstrip('0') for c in code]
  if isinstance(code, str):
    codes=codes[0]
  return codes

# Notations

## expand hyphen

In [0]:
#export
# function to expand a string like 'K51.2-K53.8' to a list of codes

# Need regex to extract the number component of the input string

# The singledispach decorator enables us to have the same name, but use
# different functions depending on the datatype of the first argument.
#
# In our case we want one function to deal with a single string input, and
# another to handle a list of strings. It could all be handled in a single
# function using nested if, but singledispatch makes it less messy and more fun!


# Here is the main function, it is just the name and an error message if the
# argument does not fit any of the inputs that wil be allowed

@singledispatch
def expand_hyphen(expr):
  """
  Expands codes expression(s) that have hyphens to list of all codes

  Args:
      code (str or list of str): String or list of strings to be expanded

  Returns:
      List of strings

  Examples:
      expand_hyphen('C00*-C26*')
      expand_hyphen('b01.1*-b09.9*')
      expand_hyphen('n02.2-n02.7')
      expand_hyphen('c00*-c260')
      expand_hyphen('b01-b09')
      expand_hyphen('b001.1*-b009.9*')
      expand_hyphen(['b001.1*-b009.9*', 'c11-c15'])
  Note:
      Unequal number of decimals in start and end code is problematic.
      Example: C26.0-C27.11 will not work since the meaning is not obvious:
      Is the step size 0.01? In which case C27.1 will not be included, while
      C27.10 will be (and traing zeros can be important in codes)
  """
  raise ValueError('The argument must be a string or a list')

# register the function to be used if the input is a string
@expand_hyphen.register(str)
def _(expr):
    # return immediately if nothing to expand
    if '-' not in expr:
      return [expr]

    lower, upper = expr.split('-')

    lower=lower.strip()

    # identify the numeric component of the code
    lower_str = re.search("\d*\.\d+|\d+", lower).group()
    upper_str = re.search("\d*\.\d+|\d+", upper).group()
    # note: what about european decimal notation?
    # also note: what if multiple groups K50.1J8.4-etc


    lower_num = int(lower_str.replace('.',''))
    upper_num = int(upper_str.replace('.','')) +1

    if upper_num<lower_num:
      raise ValueError('The start code cannot have a higher number than the end code')

    # remember length in case of leading zeros
    length = len(lower_str)

    nums = range(lower_num, upper_num)

    # must use integers in a loop, not floats
    # which also means that we must multiply and divide to get decimal back
    # and take care of leading and trailing zeros that may disappear
    if '.' in lower_str:
      lower_decimals = len(lower_str.split('.')[1])
      upper_decimals = len(upper_str.split('.')[1])
      if lower_decimals==upper_decimals:
        multiplier = 10**lower_decimals
        codes = [lower.replace(lower_str, format(num /multiplier, f'.{lower_decimals}f').zfill(length)) for num in nums]
      # special case: allow k1.1-k1.123, but not k.1-k2.123 the last is ambigious: should it list k2.0 only 2.00?
      elif (lower_decimals<upper_decimals) & (upper_str.split('.')[0]==lower_str.split('.')[0]):
        from_decimal = int(lower_str.split('.')[1])
        to_decimal = int(upper_str.split('.')[1]) +1
        nums = range(from_decimal, to_decimal)
        decimal_str = '.'+lower.split('.')[1]
        codes = [lower.replace(decimal_str, '.'+str(num)) for num in nums]
      else:
        raise ValueError('The start code and the end code do not have the same number of decimals')
    else:
        codes = [lower.replace(lower_str, str(num).zfill(length)) for num in nums]
    return codes


# register the function to be used if if the input is a list of strings
@expand_hyphen.register(list)
def _(expr):
  extended = []
  for word in expr:
    extended.extend(expand_hyphen(word))
  return extended

## expand star

In [0]:
#export
# A function to expand a string with star notation (K50*)
# to list of all codes starting with K50

@singledispatch
def expand_star(code, all_codes=None):
  """
  Expand expressions with star notation to a list of all values with the specified pattern

  Args:
    expr (str or list): Expression (or list of expressions) to be expanded
    all_codes (list) : A list of all codes

  Examples:
    expand_star('K50*', all_codes=icd9)
    expand_star('K*5', all_codes=icd9)
    expand_star('*5', all_codes=icd9)

  """
  raise ValueError('The argument must be a string or a list')

@expand_star.register(str)
def _(code, all_codes=None):
  # return immediately if there is nothing to expand
  if '*' not in code:
    return [code]

  start_str, end_str = code.split('*')

  if start_str and end_str:
    codes = {code for code in all_codes if (code.startswith(start_str) & code.endswith(end_str))}

  if start_str:
    codes = {code for code in all_codes if code.startswith(start_str)}

  if end_str:
    codes = {code for code in all_codes if code.endswith(end_str)}

  return sorted(list(codes))

@expand_star.register(list)
def _(code, all_codes=None):

  expanded=[]
  for star_code in code:
    new_codes = expand_star(star_code, all_codes=all_codes)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)

## expand colon

In [0]:
#export
# function to get all codes in a list between the specified start and end code
# Example: Get all codes between K40:L52

@singledispatch
def expand_colon(code, all_codes=None):
  raise ValueError('The argument must be a string or a list')

@expand_colon.register(str)
def _(code, all_codes=None):
  """
  Expand expressions with colon notation to a list of complete code names
  code (str or list): Expression (or list of expressions) to be expanded
  all_codes (list or array) : The list to slice from

  Examples
    K50:K52
    K50.5:K52.19
    A3.0:A9.3

  Note: This is different from hyphen and star notation because it can handle
  different code lengths and different number of decimals

  """
  if ':' not in code:
    return [code]

  startstr, endstr = code.split(':')

  # remove spaces
  startstr = startstr.strip()
  endstr =endstr.strip()

  # find start and end position
  startpos = all_codes.index(startstr)
  endpos = all_codes.index(endstr) + 1

  # slice list
  expanded = all_codes[startpos:endpos+1]

  return expanded


@expand_colon.register(list)
def _(code, all_codes=None, regex=False):
  expanded=[]

  for cod in code:
    new_codes = expand_colon(cod, all_codes=all_codes)
    expanded.extend(new_codes)

  return expanded

## expand regex

In [0]:
#export
# Return all elements in a list that fits a regex pattern

@singledispatch
def expand_regex(code, all_codes):
  raise ValueError('The argument must be a string or a list of strings')

@expand_regex.register(str)
def _(code, all_codes=None):
  code_regex = re.compile(code)
  expanded = {code for code in all_codes if code_regex.match(code)}
  # uniqify
  expanded = list(set(expanded))
  return expanded

@expand_regex.register(list)
def _(code, all_codes):
  expanded=[]

  for cod in code:
    new_codes = expand_regex(cod, all_codes=all_codes)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = sorted(list(set(expanded)))

  return expanded

## expand code

In [0]:
#export
@singledispatch
def expand_code(code, all_codes=None,
                hyphen=True, star=True, colon=True, regex=False,
                drop_dot=False, drop_leading_zero=False,
                sort_unique=True):
  raise ValueError('The argument must be a string or a list of strings')

@expand_code.register(str)
def _(code, all_codes=None,
      hyphen=True, star=True, colon=True, regex=False,
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):
  #validating input
  if (not regex) and (':' in code) and (('-' in code) or ('*' in code)):
    raise ValueError('Notation using colon must start from and end in specific codes, not codes using star or hyphen')

  if regex:
    codes = expand_regex(code, all_codes=all_codes)
    return codes

  if drop_dot:
    code = del_dot(code)

  codes=[code]

  if hyphen:
    codes=expand_hyphen(code)
  if star:
    codes=expand_star(codes, all_codes=all_codes)
  if colon:
    codes=expand_colon(codes, all_codes=all_codes)

  if sort_unique:
    codes = sorted(list(set(codes)))

  return codes

@expand_code.register(list)
def _(code, all_codes=None, hyphen=True, star=True, colon=True, regex=False,
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):

  expanded=[]

  for cod in code:
    new_codes = expand_code(cod, all_codes=all_codes, hyphen=hyphen, star=star, colon=colon, regex=regex, drop_dot=drop_dot, drop_leading_zero=drop_leading_zero)
    expanded.extend(new_codes)

  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)

## expand columns

In [0]:
#export
def expand_columns(expr, all_columns=None, df=None, star=True,
                   hyphen=True, colon=True, regex=None, info=None):
    """
    Expand columns with special notation to their full column names

    """
    notations = '* - :'.split()
    # return immediately if not needed
    if not any(symbol in expr for symbol in notations):
      return [expr]

    # get a list of columns of it is only implicity defined by the df
    # warning: may depreciate this, require explicit all_columns
    if df & (not all_columns):
      all_columns=list(df.columns)

    if regex:
      cols = [col for col in all_columns if re.match(regex, expr)]
    else:
      if hyphen:
        cols = expand_hyphen(expr)
      if star:
        cols = expand_star(expr, all_codes=all_columns)
      if colon:
        cols = expand_colon(expr, all_codes=all_columns)

    return cols

# More helper functions


## get rows

In [0]:
#export
# mark rows that contain certain codes in one or more colums
def get_rows(df, codes, cols=None, sep=None, pid='pid', info=None, fix=True):
  """
  Make a boolean series that is true for all rows that contain the codes

  Args
    df (dataframe or series): The dataframe with codes
    codes (str, list, set, dict): codes to be counted
    cols (str or list): list of columns to search in
    sep (str): The symbol that seperates the codes if there are multiple codes in a cell
    pid (str): The name of the column with the personal identifier

  """

  # check if evaluated previously
  info, rows = memory(info=info, func = 'get_rows', expr=codes)
  if rows:
    return rows

  # check if codes and columns need to be expanded (needed if they use notation)
  if fix:
    # do this when if cols exist, but if it does not ...
    cols = expand_columns(expr=cols, all_columns=list(df.columns), info=info)
    all_codes = sorted(unique(df=df, cols=cols, sep=sep))
    codes = expand_code(codes, all_codes=all_codes)

  # codes and cols should be lists
  codes = listify(codes)
  cols = listify(cols)

  # approach depends on whether we have multi-value cells or not
  # if sep exist, then have multi-value cells
  if sep:
    # have multi-valued cells
    # note: this assumes the sep is a regex word delimiter
    codes = [rf'\b{code}\b' for code in codes]
    codes_regex = '|'.join(codes)

    # starting point: no codes have been found
    # needed since otherwise the function might return None if no codes exist
    rows = pd.Series(False*len(df),index=df.index)

   # loop over all columns and mark when a code exist
    for col in cols:
      rows=rows | df[col].str.contains(codes_regex, na=False)

  # if not multi valued cells
  else:
    mask = df[cols].isin(codes)
    rows = mask.any(axis=1)
  return rows

## make codes

In [0]:
#export
def make_codes(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes

  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  # each code is assumed to consist of a letter and a number
  alphabet = list('abcdefghigjklmnopqrstuvwxyz')
  letters=alphabet[:letters+1]

  # make random numbers same if seed is specified
  if seed:
    np.random.seed(0)

  # determine the number of codes to be drawn for each event
  n_codes=np.random.negative_binomial(1, p=0.3, size=n)
  # avoid zero (all events have to have at least one code)
  n_codes=n_codes+1

  # for each event, randomly generate a the number of codes specified by n_codes
  codes=[]
  for i in n_codes:
      diag = [np.random.choice(letters).upper()+
              str(int(np.random.uniform(low=1, high=numbers)))
              for num in range(i)]

      code_string=','.join(diag)
      codes.append(code_string)

  # create a dataframe based on the list
  df=pd.DataFrame(codes)
  df.columns=['code']

  return df

In [0]:
make_codes(10)

## make data

In [0]:
#export
def make_data(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes

  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  pid = range(n)
  df_person=pd.DataFrame(index = pid)

  #female = np.random.binomial(1, 0.5, size =n)
  gender = np.random.choice(['male', 'female'], size=n)
  region = np.random.choice(['north', 'south', 'east', 'west'], size=n)
  birth_year = np.random.randint(1920, 1980, size=n)
  birth_month = np.random.randint(1,12, size=n)
  birth_day = np.random.randint(1,28, size=n) # ok, I know!
  events_per_year = np.random.poisson(1, size=n)
  years = 2020 - birth_year
  events = years * events_per_year
  events = np.where(events==0,1,events)
  events = events.astype(int)
  all_codes=[]
  codes = [all_codes.extend(make_codes(n=n, letters=letters,
                                       numbers=numbers,
                                       seed=seed)['code'].tolist())
          for n in events]

  days_alive = (2020 - birth_year) *365

  days_and_events = zip(days_alive.tolist(), events.tolist())
  all_days=[]
  days_after_birth = [all_days.extend(np.random.randint(0, max_day, size=n)) for max_day, n in days_and_events]
  pid_and_events = zip(list(pid), events.tolist())
  all_pids=[]
  pids = [all_pids.extend([p+1]*e) for p, e in pid_and_events]

  df_events = pd.DataFrame(index=all_pids)
  df_events['codes'] = all_codes
  df_events['days_after'] = all_days

  #df_person['female'] = female
  df_person['gender'] = gender

  df_person['region'] = region
  df_person['year'] = birth_year
  df_person['month'] = birth_month
  df_person['day'] = birth_day
  df = df_events.merge(df_person, left_index=True, right_index=True)
  df['birth_date'] = pd.to_datetime(df[['year', 'month', 'day']])
  df['event_date'] = df['birth_date'] + pd.to_timedelta(df.days_after, unit='d')
  del df['month']
  del df['day']
  del df['days_after']
  df['pid'] = df.index
  df.index_name = 'pid_index'
  df=df[['pid', 'gender', 'birth_date', 'event_date', 'region', 'codes']]
  # include deaths too?
  return df

In [0]:
#df = make_data(n=1000)

In [0]:
#df

In [0]:
#count_person('max 2 L35')
#count_person('x before y')

# formatting an expression

## insert_external

In [0]:
#export
def insert_external(expr):
  """
  Replaces variables prefixed with @ in the expression with the
  value of the variable from the global namespace

  Example:
      x=['4AB02', '4AB04', '4AB06']
      expr = '@x before 4AB02'
      insert_external(expr)
  """
  externals = [word.strip('@') for word in expr.split() if word.startswith('@')]
  for external in externals:
      tmp = globals()[external]
      expr = expr.replace(f'@{external} ', f'{tmp} ')
  return expr

In [0]:
  x_1=['4AB02', '4AB04', '4AB06']
  expr = '@x_1 before 4AB02'
  insert_external(expr)

# get inpatient data

To test the functions and to calculate the Charslon index we need some data. Here we will use data on hospital visits from Medicare: 


In [0]:
# Use pandas
import pandas as pd

In [0]:
# Read synthetic medicare sample data on inpatient hospital stays
path = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/'
inpatient_file = 'DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip'

inpatient = pd.read_csv(path+inpatient_file)

inpatient.columns = inpatient.columns.str.lower()
# easier to use a column called 'pid' than 'desynpuf_id'
inpatient['pid']=inpatient['desynpuf_id']

#set index to the personal id, but also keep id as a column
inpatient = inpatient.set_index('pid', drop=False)
inpatient.index.name='pid_index'


In [0]:
# Have a look
inpatient.head()

In [0]:
# make a list of columns with information about diagnostic codes
icd_cols = list(inpatient.columns[inpatient.columns.str.startswith('icd9_dgns_cd')])
icd_cols

Make a list of all unique ICD9 codes that exist, a all_codes:

In [0]:
# Codes to calculate CCI using ICD-9 (CM, US, Enhanced)
# Source: http://mchp-appserv.cpe.umanitoba.ca/concept/Charlson%20Comorbidities%20-%20Coding%20Algorithms%20for%20ICD-9-CM%20and%20ICD-10.pdf

infarction = '''
      410* 
      412*
      '''

heart_failure = '''
        390.91 
        402.21 402.11 402.91 
        404.01 404.03 404.11 404.13 404.91 404.93 
        425.4-425.9 
        428*
        '''

peripheral_vascular = '''
        093.0
        437.3
        440*
        441*
        443.1-443.9
        447.1
        557.1 557.9
        V43.4
        '''

cerebrovascular = '''
        362.34
        430*-438*
        '''
dementia = '''
        290*
        294.1
        331.2
        '''

pulmonary ='''
      416.8 416.9
      490*-505* 
      506.4
      508.1 508.8
      '''
rheumatic = '''
      446.5
      710.0-710.4
      714.0-714.2 714.8
      725*
      '''

peptic_ulcer = '531*-534*'

liver_mild ='''
      070.22
      070.23
      070.32
      070.33
      070.44
      070.54
      070.6
      070.9
      570.*
      571.*
      573.3 573.4 573.8 573.9
      V42.7
      '''
# Interesting, diabetes seems to be 5 digits long in the data, but not the specified codes
diabetes_without_complication = '250.0*-250.3* 250.8* 250.9*'

diabetes_with_complication = '250.4*-250.7*'

plegia = '''
    334.1
    342.*
    343.*
    344.0-344.6
    344.9
    '''

renal = '''
    403.01 403.11,403.91 
    404.02 404.03 404.12 404.13 404.92 404.93
    582.*  
    583.0-583.7
    585*
    586*
    588.0
    V42.0
    V45.1
    V56*
    '''

malignancy = '''
    140*-172*
    174.0-195.8
    200*-208*
    238.6
    '''

liver_not_mild = '''
    456.0-456.2
    572.2-572.8
    '''

tumor = '196*-199*'

hiv = '042*-044*'

Put all the strings that describe the codes for the comorbitities in a single datastructure:

In [0]:
icd9 = unique(df=inpatient, cols = icd_cols, all_str=True)

In [0]:
# A dictionary with names of cormobitities and the associated medical codes

code_string = { 'infarction' : infarction, 
               'heart_failure' : heart_failure, 
               'peripheral_vascular' : peripheral_vascular, 
               'cerebrovascular' : cerebrovascular, 
               'dementia' : dementia, 
               'pulmonary' : pulmonary, 
               'rheumatic' : rheumatic, 
               'peptic_ulcer' : peptic_ulcer, 
               'liver_mild' : liver_mild, 
               'diabetes_without_complication' : diabetes_without_complication, 
               'diabetes_with_complication' : diabetes_with_complication, 
               'plegia' : plegia, 
               'renal' : renal, 
               'malignancy' : malignancy, 
               'liver_not_mild' : liver_not_mild, 
               'tumor' : tumor, 
               'hiv' : hiv}

Having created a all_codes, we can use the functions we have created to expand the description for all the different comorbidities to include all the specific codes:

In [0]:
codes = {disease : expand_code(codes.split(), 
                               all_codes=icd9,
                               drop_dot=True,
                               drop_leading_zero=True) 
        for disease, codes in code_string.items()}

And we can check if it really expanded the codes, for instance by examining the codes for mild liver disease:

In [0]:
codes['liver_mild']

In order to do the calculations, we need the weights associated with each comorbidity. These weights are related to the predictive power of the comorbididy for the probability of dying in a given time period. There are a few different standards, but with relatively minor varitions. Here we use the following:


In [0]:
charlson_points = { 'infarction': 1, 
                   'heart_failure': 1, 
                   'peripheral_vascular': 1, 
                   'cerebrovascular': 1, 
                   'dementia': 1, 
                   'pulmonary': 1, 
                   'rheumatic': 1, 
                   'peptic_ulcer': 1, 
                   'liver_mild': 1, 
                   'diabetes_without_complication': 1, 
                   'diabetes_with_complication': 2, 
                   'plegia': 2, 
                   'renal': 2, 
                   'malignancy': 2, 
                   'liver_not_mild': 3, 
                   'tumor': 6, 
                   'hiv': 6}


We also need the function that takes a set of codes and identifies the rows and persons who have the codes (a function we developed in a previous notebook):

In [ ]:
#hide
from nbdev.showdoc import *

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
Converted query_language.ipynb.


In [2]:
import nbdev

In [3]:
from nbdev.sync import script2notebook 

In [ ]:
script2not